In [6]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

load_dotenv()
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

print(" Environment loaded")

 Environment loaded


In [7]:
import json
from typing import List, Dict
from datetime import datetime
from tqdm.auto import tqdm

# LangChain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

# Pinecone
from pinecone import Pinecone, ServerlessSpec

# PDF processing
import pypdf

print(" All imports successful")

 All imports successful


### 1. Load Configuration from setup_and_data_exploration.ipynb


In [8]:
# Load Phase 1 configuration
config_path = project_root / "phase1_config.json"

if config_path.exists():
    with open(config_path, 'r') as f:
        phase1_config = json.load(f)
    
    CHUNK_SIZE = phase1_config['chunking']['size']
    CHUNK_OVERLAP = phase1_config['chunking']['overlap']
    
    print(" Loaded Phase 1 config:")
    print(f"   Chunk size: {CHUNK_SIZE}")
    print(f"   Overlap: {CHUNK_OVERLAP}")
else:
    print("  Phase 1 config not found, using defaults")
    CHUNK_SIZE = 800
    CHUNK_OVERLAP = 150

 Loaded Phase 1 config:
   Chunk size: 800
   Overlap: 150


### 2. Initialize HuggingFace Embeddings

In [9]:
# Initialize HuggingFace Embeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Test embedding
test_text = "This is a test sentence for embeddings"
test_embedding = embeddings_model.embed_query(test_text)

print(" Test embedding generated:")
print(f"   Dimension: {len(test_embedding)}")
print(f"   Sample values: {test_embedding[:5]}")

 Test embedding generated:
   Dimension: 384
   Sample values: [0.015511015430092812, -0.004481229465454817, 0.07071685045957565, 0.04283339902758598, 0.043629080057144165]


### 3. Setup Pinecone

In [10]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME", "agri-chatbot")

print("✅ Pinecone API key found")
print(f"📌 Index name: {PINECONE_INDEX_NAME}")

✅ Pinecone API key found
📌 Index name: agri-chatbot


In [11]:
# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check existing indexes
existing_indexes = pc.list_indexes().names()
print(f"Existing indexes: {existing_indexes if existing_indexes else 'None'}")

Existing indexes: None


In [18]:
EMBEDDING_DIM = 384  # all-MiniLM-L6-v2 dimension

# Index names
DISEASE_INDEX_NAME = f"{PINECONE_INDEX_NAME}-disease"
SCHEME_INDEX_NAME = f"{PINECONE_INDEX_NAME}-scheme"

existing_indexes = pc.list_indexes().names()

# Create Disease Index
if DISEASE_INDEX_NAME not in existing_indexes:
    print(f"🔨 Creating disease index: {DISEASE_INDEX_NAME}")
    pc.create_index(
        name=DISEASE_INDEX_NAME,
        dimension=EMBEDDING_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f" Disease index created!")
else:
    print(f"Disease index exists: {DISEASE_INDEX_NAME}")

# Create Scheme Index
if SCHEME_INDEX_NAME not in existing_indexes:
    print(f"🔨 Creating scheme index: {SCHEME_INDEX_NAME}")
    pc.create_index(
        name=SCHEME_INDEX_NAME,
        dimension=EMBEDDING_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f" Scheme index created!")
else:
    print(f"Scheme index exists: {SCHEME_INDEX_NAME}")

# Wait for indexes to be ready
print(" Waiting for indexes to initialize (30 seconds)...")
import time
time.sleep(30)

# Connect to both indexes
disease_index = pc.Index(DISEASE_INDEX_NAME)
scheme_index = pc.Index(SCHEME_INDEX_NAME)

print(f"\n Connected to both indexes")

🔨 Creating disease index: agri-chatbot-disease
 Disease index created!
🔨 Creating scheme index: agri-chatbot-scheme
 Scheme index created!
 Waiting for indexes to initialize (30 seconds)...

 Connected to both indexes


In [19]:
# Check Disease Index
disease_stats = disease_index.describe_index_stats()
print(f"Disease Index Statistics:")
print(f"   Name: {DISEASE_INDEX_NAME}")
print(f"   Vectors: {disease_stats.total_vector_count}")
print(f"   Dimension: {disease_stats.dimension}")

# Check Scheme Index
scheme_stats = scheme_index.describe_index_stats()
print(f"\nScheme Index Statistics:")
print(f"   Name: {SCHEME_INDEX_NAME}")
print(f"   Vectors: {scheme_stats.total_vector_count}")
print(f"   Dimension: {scheme_stats.dimension}")

Disease Index Statistics:
   Name: agri-chatbot-disease
   Vectors: 0
   Dimension: 384

Scheme Index Statistics:
   Name: agri-chatbot-scheme
   Vectors: 0
   Dimension: 384


### 4. Load and Process PDFs

In [20]:
def load_pdf(pdf_path: Path) -> List[Dict]:
    """Load PDF and return list of page documents"""
    documents = []
    
    with open(pdf_path, 'rb') as file:
        pdf_reader = pypdf.PdfReader(file)
        
        for page_num in tqdm(range(len(pdf_reader.pages)), desc=f"Loading {pdf_path.name}"):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            
            if text and text.strip():
                documents.append({
                    "page_number": page_num + 1,
                    "text": text,
                    "source": pdf_path.name
                })
    
    return documents

print(" PDF loader ready")

 PDF loader ready


In [21]:
DATA_DIR = project_root / "data"
DISEASE_PDF = DATA_DIR / "CitrusPlantPestsAndDiseases.pdf"
SCHEME_PDF = DATA_DIR / "GovernmentSchemes.pdf"

# Load Disease PDF
disease_docs = []
if DISEASE_PDF.exists():
    disease_docs = load_pdf(DISEASE_PDF)
    print(f"Disease PDF: {len(disease_docs)} pages")
else:
    print("Disease PDF not found")

# Load Scheme PDF
scheme_docs = []
if SCHEME_PDF.exists():
    scheme_docs = load_pdf(SCHEME_PDF)
    print(f"Scheme PDF: {len(scheme_docs)} pages")
else:
    print("Scheme PDF not found")

Loading CitrusPlantPestsAndDiseases.pdf:   0%|          | 0/67 [00:00<?, ?it/s]

Disease PDF: 67 pages


Loading GovernmentSchemes.pdf:   0%|          | 0/63 [00:00<?, ?it/s]

Scheme PDF: 63 pages


### 5. Create Chunks with Metadata

In [22]:
def create_chunks(docs: List[Dict], doc_type: str) -> List[Document]:
    """Create chunks with metadata"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    all_chunks = []
    for doc in docs:
        chunks = text_splitter.split_text(doc["text"])
        
        for i, chunk in enumerate(chunks):
            all_chunks.append(Document(
                page_content=chunk,
                metadata={
                    "source": doc["source"],
                    "page_number": doc["page_number"],
                    "chunk_id": len(all_chunks),
                    "doc_type": doc_type,
                    "chunk_index": i
                }
            ))
    
    return all_chunks

print(" Chunking function ready")

 Chunking function ready


In [17]:
# Create disease chunks
disease_chunks = []
if disease_docs:
    disease_chunks = create_chunks(disease_docs, "disease")
    print(f"Disease chunks: {len(disease_chunks)}")

# Create scheme chunks
scheme_chunks = []
if scheme_docs:
    scheme_chunks = create_chunks(scheme_docs, "scheme")
    print(f"Scheme chunks: {len(scheme_chunks)}")

total_chunks = len(disease_chunks) + len(scheme_chunks)
print(f"\nTotal chunks to upload: {total_chunks}")

Disease chunks: 296
Scheme chunks: 130

Total chunks to upload: 426


###  6. Upload to Pinecone

In [23]:
def upload_to_pinecone(chunks: List[Document], target_index, index_name: str, batch_size: int = 100):
    """Upload chunks to specific Pinecone index"""
    print(f"\n Uploading {len(chunks)} chunks to '{index_name}'...")
    
    for i in tqdm(range(0, len(chunks), batch_size), desc="Uploading"):
        batch = chunks[i:i + batch_size]
        
        # Generate embeddings
        texts = [chunk.page_content for chunk in batch]
        embeddings = embeddings_model.embed_documents(texts)
        
        # Prepare vectors
        vectors = []
        for j, (chunk, embedding) in enumerate(zip(batch, embeddings)):
            vector_id = f"{index_name}_{i+j}"
            vectors.append({
                "id": vector_id,
                "values": embedding,
                "metadata": {
                    **chunk.metadata,
                    "text": chunk.page_content[:1000]
                }
            })
        
        # Upload to specific index (no namespace needed)
        target_index.upsert(vectors=vectors)
    
    print(f" Upload complete!")

print(" Upload function ready")

 Upload function ready


In [24]:
if disease_chunks:
    upload_to_pinecone(
        disease_chunks, 
        disease_index, 
        DISEASE_INDEX_NAME,
        batch_size=50
    )


 Uploading 296 chunks to 'agri-chatbot-disease'...


Uploading:   0%|          | 0/6 [00:00<?, ?it/s]

 Upload complete!


In [25]:
if scheme_chunks:
    upload_to_pinecone(
        scheme_chunks, 
        scheme_index, 
        SCHEME_INDEX_NAME,
        batch_size=50
    )


 Uploading 130 chunks to 'agri-chatbot-scheme'...


Uploading:   0%|          | 0/3 [00:00<?, ?it/s]

 Upload complete!


In [26]:
# Refresh stats for both indexes
disease_stats = disease_index.describe_index_stats()
scheme_stats = scheme_index.describe_index_stats()

print(f"\n Final Index Statistics:")
print(f"\n Disease Index ({DISEASE_INDEX_NAME}):")
print(f"   Total vectors: {disease_stats.total_vector_count}")

print(f"\n Scheme Index ({SCHEME_INDEX_NAME}):")
print(f"   Total vectors: {scheme_stats.total_vector_count}")

print(f"\n Combined Total: {disease_stats.total_vector_count + scheme_stats.total_vector_count} vectors")


 Final Index Statistics:

 Disease Index (agri-chatbot-disease):
   Total vectors: 296

 Scheme Index (agri-chatbot-scheme):
   Total vectors: 130

 Combined Total: 426 vectors


### 7. Test Semantic Search

In [27]:
def search_index(query: str, target_index, index_name: str, top_k: int = 3):
    """Search specific Pinecone index"""
    print(f"\n🔍 Query: '{query}'")
    print(f"📂 Index: {index_name}\n")
    
    # Generate query embedding
    query_embedding = embeddings_model.embed_query(query)
    
    # Search specific index
    results = target_index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    # Display results
    for i, match in enumerate(results.matches, 1):
        print(f"Result {i} (Score: {match.score:.4f}):")
        print(f"  Page: {match.metadata.get('page_number')}")
        print(f"  Text: {match.metadata.get('text', '')[:150]}...")
        print()
    
    return results

print(" Search function ready")

 Search function ready


In [28]:
if disease_chunks:
    search_index(
        query="What are the symptoms of citrus canker?",
        target_index=disease_index,
        index_name=DISEASE_INDEX_NAME,
        top_k=3
    )


🔍 Query: 'What are the symptoms of citrus canker?'
📂 Index: agri-chatbot-disease

Result 1 (Score: 0.7156):
  Page: 57
  Text: Below is a consolidated table summarizing the major issues covered, for quick reference. It lists 
the problem, causal agent/category, key symptoms, a...

Result 2 (Score: 0.6981):
  Page: 24
  Text: structure. - Note distribution: often starts on windward side of tree and lower canopy (wind-rain 
hits there). - Test: If you put an infected leaf in...

Result 3 (Score: 0.6852):
  Page: 21
  Text: Category: Bacterial disease (leaf, fruit, twig canker) 
Description: Citrus canker is a highly contagious disease caused by the bacterium Xanthomonas ...



In [29]:
if scheme_chunks:
    search_index(
        query="What subsidies are available for drip irrigation?",
        target_index=scheme_index,
        index_name=SCHEME_INDEX_NAME,
        top_k=3
    )


🔍 Query: 'What subsidies are available for drip irrigation?'
📂 Index: agri-chatbot-scheme

Result 1 (Score: 0.6103):
  Page: 13
  Text: Farmer
Category
Farm
Size
Limit
Drip
Irrigation
Subsidy
Sprinkle
r Subsidy
Max.
Assistance
SC/ST
12.5
acres
max
55-100% 50-90%
Rs. 5,61,185
(HIGHEST I...

Result 2 (Score: 0.5655):
  Page: 2
  Text: only
Joint ownership: NOC from all co-owners required
Tenant cultivation: Documented proof of landlord's consent
2. Farmer Categories (Priority Order)...

Result 3 (Score: 0.5539):
  Page: 13
  Text: Pump & motor (0.5 HP) Rs. 8,000 Rs. 10,000
Installation labor Rs. 5,000 Rs. 4,000
Filter unit Rs. 3,000 Rs. 2,000
Total Investment Rs. 60,000 Rs. 32,0...



In [31]:
if disease_chunks and scheme_chunks:
    print("🔍 Testing hybrid query (searches both indexes):\n")
    
    query = "Government help for citrus greening disease"
    
    print("=" * 80)
    print("Searching DISEASE index:")
    print("=" * 80)
    disease_results = search_index(query, disease_index, DISEASE_INDEX_NAME, top_k=2)
    
    print("\n" + "=" * 80)
    print("Searching SCHEME index:")
    print("=" * 80)
    scheme_results = search_index(query, scheme_index, SCHEME_INDEX_NAME, top_k=2)
    
    print("\n For hybrid queries, we combine results from both indexes!")

🔍 Testing hybrid query (searches both indexes):

Searching DISEASE index:

🔍 Query: 'Government help for citrus greening disease'
📂 Index: agri-chatbot-disease

Result 1 (Score: 0.7058):
  Page: 67
  Text: [65] [66] File:Citrus greening UGA5201066.jpg - Wikimedia Commons 
https://commons.wikimedia.org/wiki/File:Citrus_greening_UGA5201066.jpg 
[67] An ove...

Result 2 (Score: 0.6956):
  Page: 1
  Text: Comprehensive Guide to Citrus Diseases, Pests, 
and Disorders in India 
Introduction 
Citrus (including sweet oranges, mandarins, lemons, and limes) i...


Searching SCHEME index:

🔍 Query: 'Government help for citrus greening disease'
📂 Index: agri-chatbot-scheme

Result 1 (Score: 0.6640):
  Page: 8
  Text: Actual yield: 4 MT/hectare (due to pest outbreak)
Loss percentage: 50%
Deductible: 20%
Indemnity = (50% - 20%) × Sum Insured
Claim = 30% × Rs. 1,40,00...

Result 2 (Score: 0.6454):
  Page: 42
  Text: Spread: Windblown rain, tool contact, insects
Management:
Remove infected plant mater

In [32]:
phase2_config = {
    "disease_index": DISEASE_INDEX_NAME,
    "scheme_index": SCHEME_INDEX_NAME,
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "embedding_dimension": EMBEDDING_DIM,
    "disease_chunks": len(disease_chunks),
    "scheme_chunks": len(scheme_chunks),
    "total_vectors": len(disease_chunks) + len(scheme_chunks),
    "approach": "separate_indexes",  # Note the approach
    "created_at": datetime.now().isoformat()
}

with open(project_root / "phase2_config.json", 'w') as f:
    json.dump(phase2_config, f, indent=2)

print("✅ Phase 2 config saved")
print(json.dumps(phase2_config, indent=2))

✅ Phase 2 config saved
{
  "disease_index": "agri-chatbot-disease",
  "scheme_index": "agri-chatbot-scheme",
  "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
  "embedding_dimension": 384,
  "disease_chunks": 296,
  "scheme_chunks": 130,
  "total_vectors": 426,
  "approach": "separate_indexes",
  "created_at": "2026-01-05T02:44:08.877967"
}
